# Methodes de descente de Gradient

In [1]:
f(x)=x^2+1
df(x)=2*x # admettons qu'on a la dérivée sinon on peut l'approximer avec les éléments finis

df (generic function with 1 method)

In [2]:
using Optim
using LinearAlgebra

In [3]:
function bracket_minimum(f, x=0; s=1e-2, k=2.0) # Cette fonction classique permet de réduire le domaine de recherche du min
    a, ya= x,f(x)
    b, yb= a+s, f(a+s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end  

bracket_minimum (generic function with 2 methods)

In [4]:
(a,c)=bracket_minimum(f)

(-0.01, 0.01)

In [5]:
function line_search(f,x,d) # Cette fonction sera utile pour modifier le pas
    obj=c->f(x+c*d)
    a, b = bracket_minimum(obj)
    c = Optim.optimize(obj, a, b)
    m=Optim.minimizer(c)
    return x + a*d
end


line_search (generic function with 1 method)

In [6]:
line_search(f,1,2)

-1.54

In [7]:
i=optimize(f, -0.5,0.5)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-0.500000, 0.500000]
 * Minimizer: 4.302114e-16
 * Minimum: 1.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

In [8]:
Optim.minimizer(i)

4.3021142204224816e-16

In [10]:
abstract type DescentMethod end
struct GradientDescent <: DescentMethod
    a
end
init!(M::GradientDescent, f, df, x) = M
function step(M::GradientDescent, f, df, x)
    a,g= M.a, df(x)
    return x-a*g
end

step (generic function with 1 method)

In [11]:
mutable struct ConjugateGradientDescent <: DescentMethod
    d
    g
end
function init!(M::ConjugateGradientDescent, f, df, x)
    M.g = df(x)
    M.d = -M.g
    return M
end


function step1(M::ConjugateGradientDescent, f, df, x)
    d,g=M.d,M.g
    dg =df(x)
    β = max(0, dot(dg, dg-g)/(g⋅g))
    d′ = -dg + β*d
    x′ = line_search(f, x, d′)
    M.d, M.g = d′, dg
    return x′
end        

step1 (generic function with 1 method)

In [84]:
M=GradientDescent(0.1)

GradientDescent(0.1)

In [85]:
i=0
s=1.
while abs(s)>0.001
   s=step(M,f,df,s)
   i=i+1
end

In [86]:
i

31

In [87]:
N=ConjugateGradientDescent(df(1),-df(1))

ConjugateGradientDescent(2, -2)

In [88]:
j=0
c=1.
while abs(c)>0.001
   c=step1(N,f,df,c)
   j=j+1
end

In [89]:
j

9